In [1]:
import pandas as pd
import os
import datetime

In [2]:
operation = "rent"
city = "lisbon"

read_path = f"../data/idealista/raw/{operation}/{city}"
save_path = f"../data/idealista/unioned/{operation}/{city}"

dataframes = []

for file in os.listdir(read_path):
    file_date = file[:10]
    if file.endswith(".csv"):
        dataframes.append(pd.read_csv(f"{read_path}/{file}", index_col='propertyCode'))
        dataframes[-1]["snapshotDate"] = datetime.datetime.strptime(file_date, "%Y-%m-%d").date()

In [3]:
# Combine the DataFrames
df = pd.concat(dataframes)

# Keep only the last occurrence of each index
df = df.loc[~df.index.duplicated(keep="last")]

# Keep only the listings that have unique values for the following columns
df = df.loc[
    ~df[
        [
            "floor",
            "propertyType",
            "size",
            "rooms",
            "bathrooms",
            "address",
            "description",
        ]
    ].duplicated(keep="last")
]



In [ ]:
df.index.is_unique

In [5]:
save_path += f"/unioned-{city}-listings-for-{operation}.csv"

In [ ]:
save_path

In [7]:
# Save the unioned DataFrame to a new CSV file
df.to_csv(save_path, index_label="propertyCode")

In [ ]:
df